#### Libraries

In [1]:
#activate the scikit environment in terminal conda activate sklearn-env 
#analysis 
import statistics
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
#from statsmodels.api import add_constant

#plotting
import plotly.express as px
import plotly.graph_objects as go
from plotly.graph_objs import Data
from plotly.subplots import make_subplots

#### Paths and functions

In [2]:
import os
from pathlib import Path
import sys

path_cwd=Path.cwd()
path_input=str(path_cwd)+'/Data_input/'
path_corr_sm = str(path_cwd) + "/Data_output(corrected)/"
sys.path.append(str(path_cwd)+'/Functions')


#### Column names 
For .csv files in Data_input

In [3]:
#WC: water content (m3/m3), T: temperature (C), EC: Saturation Extract (mS/cm),Battery percent (%), Battery Voltage(mV)
SOIL_cols_1=['Date','WC1','T1','EC1','WC2','T2','EC2','WC3','T3','EC3','BAT%','BATV']
SOIL_cols_2=['Date','WC1','T1','EC1','WC2','T2','EC2' ,'BAT%','BATV']

#### Inputfiles
- List of tuples (str,str,dict): **(name,tree,{'Type':numerical_value,'Site':numerical_value})**

    - name: sensor name (e.g'z122.csv')
    - tree: name for tree (e.g DF27US)
    - Type: 2 or 3 depending on number of sensors (only 42 has 2 sensors)
    - Site: 1: upper cliff, 2: lower cliff, 3: glacial till

In [4]:
from csv import DictReader, reader

with open(path_input+'inputfiles.csv', encoding='utf-8-sig') as read_obj:
    dict_reader = DictReader(read_obj)
    list_of_dict = list(dict_reader)

    name_=[[list_of_dict[i].pop(key) for i in range(len(list_of_dict))]for key in ["Filename","Tree"]] #extract string columns: Filename,Tree, Date (if applicable) .pop(key):removes specified element from list or dict 

    res = [dict([key, int(value)] for key, value in dicts.items()) for dicts in list_of_dict]#this will make all values integers ONLY WORKS if all values in file are numerical
    
    inputfiles=[(name_[0][i],name_[1][i],res[i]) for i in range(len(list_of_dict))] #list of tuples: [(name,tree,date,{Dict})]

In [5]:
inputfiles


[('z122.csv', '27', {'Type': 3, 'Site': 1}),
 ('z123.csv', '01', {'Type': 3, 'Site': 1}),
 ('z1558.csv', '42', {'Type': 2, 'Site': 1}),
 ('z1549.csv', '03', {'Type': 3, 'Site': 1}),
 ('z129.csv', '50', {'Type': 3, 'Site': 2}),
 ('z1573.csv', '21', {'Type': 3, 'Site': 2}),
 ('z1547.csv', '48', {'Type': 3, 'Site': 3}),
 ('z1544.csv', '49', {'Type': 3, 'Site': 3})]

#### Main

- **Reading**
- **Cleaning**
- **Imputation of missing values (lr between complete and incomplete sensors within instruments)**

##### Imputation same site

In [6]:
name_sites=[]
df_sites=[]

for i,(name,tree,options) in enumerate(inputfiles):
    if options['Type'] == 2:
        soil_2=pd.read_csv(path_input+name, header=None, skiprows=2, names=SOIL_cols_2, engine='python', encoding='iso-8859-1')
        soil_2['Date'] = pd.to_datetime(soil_2['Date'], errors='coerce')
        soil_2 = soil_2.set_index('Date').resample('1H').first().loc['2022-05-26 00:00:00':'2022-10-19 00:00:00']
        soil_2['S1'], soil_2['S2'] = [soil_2[col].astype(float).where(soil_2[col].astype(float) <= 1, np.nan) for col in ['WC1', 'WC2']]
        df_2 = soil_2[['S1', 'S2']].copy()

        # Case 1: No action for rows where all three columns are NaN
        case1_mask_2 = df_2[["S1", "S2"]].isnull().all(axis=1)
        df_2.loc[case1_mask_2, ["S1", "S2"]] = np.nan
        # case 1.1: if 24 rows missing or less interpolate linearly
        if case1_mask_2.sum() <= 24:
            df_2 = df_2.interpolate(method="linear", limit_direction="both", axis=0)

        #Exploration plots of raw data
        fig = make_subplots(rows=2, cols=1)
        #fig.append_trace(go.Scatter(x=soil_m.index,y=soil_m['S1'],name="Sensor1_original"), row=1, col=1)
        fig.append_trace(go.Scatter(x=df_2.index,y=df_2['S1'],name="Sensor1_interpolate"), row=1, col=1)
        #fig.append_trace(go.Scatter(x=soil_m.index,y=soil_m['S2'],name="Sensor2_original"), row=3, col=1)
        fig.append_trace(go.Scatter(x=df_2.index,y=df_2['S2'],name="Sensor2_int"), row=2, col=1)
        fig.update_layout(height=900, width=1800, title_text="Soil_moisture"+'_'+tree+'_'+'original_2_sensors')
        fig.show()
    

    if tree == '42':
        name_sites.append(tree+'_US')
        df_sites.append(df_2)


    if options['Type'] == 3: 
        df_=pd.read_csv(path_input+name, header=None, skiprows=1, names=SOIL_cols_1, engine='python', encoding='iso-8859-1') 
        df_=df_[~df_['Date'].str.contains("[a-zA-Z]").fillna(False)].dropna(subset=['Date'])
        df_['Date'] = pd.to_datetime(df_['Date'], errors='coerce')
        df_copy = df_.copy()
        df_copy = df_copy.set_index('Date').resample('1H').first().loc['2022-05-26 00:00:00':'2022-10-19 00:00:00']
        df_copy['S1'], df_copy['S2'], df_copy['S3'] = [df_copy[col].astype(float).where(df_copy[col].astype(float) <= 1, np.nan) for col in ['WC1', 'WC2', 'WC3']]
        soil_m = df_copy[['S1', 'S2', 'S3']].copy()

        df_c = soil_m.copy()

        # Case 1: No action for rows where all three columns are NaN
        case1_mask = df_c[["S1", "S2", "S3"]].isnull().all(axis=1)
        df_c.loc[case1_mask, ["S1", "S2", "S3"]] = np.nan #another way np.where(case1_mask, np.nan, df_c[["S1", "S2", "S3"]])

        # case 1.1: if 24 rows missing or less interpolate linearly
        if case1_mask.sum() <= 24:
            df_c = df_c.interpolate(method="linear", limit_direction="both", axis=0)

        # Case 2: Linear regression for sections with one complete column and two NaN columns
        case2_mask = df_c[["S1", "S2", "S3"]].isnull().sum(axis=1) == 2 # look for rows with 2 NaNs
        case2_data = df_c.loc[case2_mask, ["S1", "S2", "S3"]].dropna(thresh=1) # df with only rows with 2 NaNs and one complete column, thresh=1: at least one non-NaN value
        if case2_mask.any(): # select dfs that have at least one row with 2 NaNs and one complete column and complete values
            complete_data = df_c.dropna(thresh=3) #thresh=3: at least 3 non-NaN values
            complete_cols = [] # should have only one element in this case
            incomplete_cols = [] # should have two elements in this case
            for col in ["S1", "S2", "S3"]:
                if case2_data[col].notnull().any():
                    complete_cols.append(col)
                if case2_data[col].isnull().any():
                    incomplete_cols.append(col)

            for icol in incomplete_cols:
                X = complete_data[complete_cols].values
                y = complete_data[icol].values
                lr = LinearRegression()
                lr.fit(X, y)
                m = lr.coef_[0]
                b = lr.intercept_
                df_c.loc[case2_mask, icol] = m * df_c.loc[case2_mask, str.join(",", complete_cols)] + b #the join is to get the column name as a string
       
        #Case 3: Linear interpolation for sections with two complete columns and one NaN column
        case3_mask = df_c[["S1", "S2", "S3"]].isnull().sum(axis=1) == 1 # look for rows with 1 NaN
        case3_data = df_c.loc[case3_mask, ["S1", "S2", "S3"]].dropna(thresh=2) # df with only rows with 1 NaN and two complete columns, thresh=2: at least two non-NaN values
        
        if case3_mask.any(): # select dfs that have at least one row with 1 NaN and two complete columns and complete values
            complete_data_ = df_c.dropna(thresh=3) #thresh=3: at least 3 non-NaN values
            complete_cols_ = [] # should have two elements in this case
            incomplete_cols_ = [] # should have only one element in this case
            for col in ["S1", "S2", "S3"]:
                if case3_data[col].notnull().any():
                    complete_cols_.append(col)
                if case3_data[col].isnull().any():
                    incomplete_cols_.append(col)

            #get average of two complete columns
            df_c['avg'] = df_c[complete_cols_].mean(axis=1)
            X_ = complete_data_[complete_cols_].mean(axis=1).values
            y_ = complete_data_[incomplete_cols_].values
            lr_ = LinearRegression()
            lr_.fit(X_.reshape(-1,1), y_)
            m_ = lr_.coef_[0]
            b_ = lr_.intercept_
            df_c.loc[case3_mask, incomplete_cols_] = m_ * df_c.loc[case3_mask, 'avg'] + b_ 

            
        # Case 4: if empty rows between non empty rows are less than 24, interpolate linear
        max_nan = 24
        
        for col in ["S1", "S2", "S3"]:
            case1=df_c[col].isna()
            case_groups = case1.ne(case1.shift()).cumsum() #create groups with dates
            case_size = case1.groupby(case_groups).transform("size") #size of each group
            for group,size in case_size.items():
                if size < max_nan +1:
                    df_c.loc[group:group+ pd.offsets.Hour(1)] = df_c.loc[group:group+ pd.offsets.Hour(1)].interpolate(method="linear", limit_direction="both", axis=0)
     
   
        # fig = make_subplots(rows=6, cols=1)
        # fig.append_trace(go.Scatter(x=df_c.index,y=df_c['S1'],name="Sensor1_imp"), row=1, col=1)
        # fig.append_trace(go.Scatter(x=soil_m.index,y=soil_m['S1'],name="Sensor1_o"), row=2, col=1)
        # fig.append_trace(go.Scatter(x=df_c.index,y=df_c['S2'],name="Sensor2_imp"), row=3, col=1)
        # fig.append_trace(go.Scatter(x=soil_m.index,y=soil_m['S2'],name="Sensor2_o"), row=4, col=1)
        # fig.append_trace(go.Scatter(x=df_c.index,y=df_c['S3'],name="Sensor3_imp"), row=5, col=1)
        # fig.append_trace(go.Scatter(x=soil_m.index,y=soil_m['S3'],name="Sensor3_o"), row=6, col=1)
        # fig.update_layout(height=900, width=1800, title_text="Soil_moisture"+'_'+tree)
        # fig.show()

    if tree == '27' or tree == '01' or tree == '03':
        name_sites.append(tree+'_US')
        df_sites.append(df_c)

    if tree == '50' or tree == '21':
        name_sites.append(tree+'_LS')
        df_sites.append(df_c)
    
    if tree == '48' or tree == '49':
        name_sites.append(tree+'_GT')
        df_sites.append(df_c)





/var/folders/73/1hl1pq915rg4htnkk3r29mym0000gn/T/ipykernel_93580/2294546654.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  soil_2['Date'] = pd.to_datetime(soil_2['Date'], errors='coerce')


In [7]:
assert False

AssertionError: 

##### Imputation all sites 

In [8]:
sensors = dict(zip(name_sites,df_sites))

In [9]:
# Step 1: Identify df with complete and incomplete time series
incomplete_dfs = []
complete_dfs = []
names_incomplete=[]
names_complete=[]

for k in sensors.keys():
    if sensors[k].isna().any().any()==True:
        incomplete_dfs.append(sensors[k])
        names_incomplete.append(k)
    else:
        complete_dfs.append(sensors[k])
        names_complete.append(k)

In [ ]:
#Step 2: Go through time series and identify segments with complete data to do regression

for incomplete_names,dfi in zip(names_incomplete,incomplete_dfs):
    r2_scores = []
    for col in dfi.columns:
        in_segment = True #flag to indicate if we are in a segment with complete data
        i0 = 0
        N = len(dfi[col].values)
        for i,val in enumerate(dfi[col].values):
            if (np.isnan(val) or i==N-1) and in_segment: 
                i1 = i-1
                for col_c in ["S1", "S2", "S3"]:
                    for complete_names,dfc in zip(names_complete,complete_dfs):
                        if col_c in dfc.columns:
                         
                            X=dfc[col_c].values[i0:i1].reshape(-1,1) 
                            y=dfi[col].values[i0:i1]

                            reg = LinearRegression().fit(X, y)
                            r2_scores.append(r2_score(y, reg.predict(X)))      

                in_segment = False
            elif not np.isnan(val) and not in_segment:
                i0 = i
                in_segment = True    

    ind= np.argmax(r2_scores)
    print(r2_scores[ind])
            

print(len(r2_scores))

In [ ]:
# Step 2: For each incomplete df, find the complete df with the highest r2 score
regressions = []
reg_names = []
for incomplete_names,incomplete_df in zip(names_incomplete,incomplete_dfs):
    r2_scores = []
    for complete_names,complete_df in zip(names_complete,complete_dfs):
        # Use only columns that have complete data
        columns_to_use = list(set(complete_df.columns) & set(incomplete_df.columns))
        X = complete_df[columns_to_use]
        y = incomplete_df[columns_to_use].reindex(X.index)  # Align the indices of X and y

        # Get overlapping non-NaN values
        valid_mask = ~X.isna().any(axis=1) & ~y.isna().any(axis=1)
        X_valid = X[valid_mask]
        y_valid = y[valid_mask]

        if len(y_valid) < 2:
            continue

        reg = LinearRegression().fit(X_valid, y_valid)
        r2_scores.append(r2_score(y_valid, reg.predict(X_valid)))

    # Choose regression with highest r2 (argmax numpy)
    best_reg = None
    best_r2 = 0
    for i, r2 in enumerate(r2_scores):
        if r2 > best_r2:
            best_reg = i
            best_r2 = r2

    regressions.append((incomplete_df, complete_dfs[best_reg]))
    reg_names.append((incomplete_names, names_complete[best_reg]))

In [ ]:
names_incomplete

In [ ]:
#this code sort of works but is missing columns
fill_results = []

for incomplete_name, incomplete_df in zip(names_incomplete, incomplete_dfs):
    best_r2 = -np.inf
    best_fill_values = None
    best_complete_name = None
    best_col_name = None
    
    for complete_name, complete_df in zip(names_complete, complete_dfs):
        for col_name in incomplete_df.columns:
            if col_name in complete_df.columns:
                incomplete_col = incomplete_df[col_name]
                complete_col = complete_df[col_name]
                
                valid_mask = ~incomplete_col.isnull() & ~complete_col.isnull()
                
                if valid_mask.any():
                    X = complete_col[valid_mask].values.reshape(-1, 1)
                    y = incomplete_col[valid_mask].values.reshape(-1, 1)
                    
                    lr = LinearRegression()
                    lr.fit(X, y)
                    y_pred = lr.predict(X)
                    
                    r2 = r2_score(y, y_pred)
                    
                    if r2 > best_r2:
                        best_r2 = r2
                        best_fill_values = lr.predict(incomplete_col[~valid_mask].index.values.astype(float).reshape(-1, 1)).flatten()
                        best_complete_name = complete_name
                        best_col_name = col_name
    
    if best_fill_values is not None:
        incomplete_col = incomplete_df[col_name]
        nan_indices = incomplete_col[incomplete_col.isnull()].index
        fill_values = np.repeat(best_fill_values, len(nan_indices))
        
        # Ensure the shapes match before assigning values
        if len(fill_values) == len(nan_indices):
            incomplete_df.loc[nan_indices, col_name] = fill_values
            fill_results.append((incomplete_name, best_complete_name, col_name))

# Print the name of the data frame and column used for filling NaN values
for result in fill_results:
    incomplete_name, complete_name, col_name = result
    print(f"Filled {col_name} in {incomplete_name} using {complete_name}")

In [ ]:
regressions = []
reg_names = []
fill_results = []
for incomplete_name, incomplete_df in zip(names_incomplete, incomplete_dfs):
    r2_scores = []
    fill_values = []
    
    for complete_name, complete_df in zip(names_complete, complete_dfs):
        for col_incomplete in incomplete_df.columns:
            if col_incomplete in complete_df.columns:
                incomplete_col = incomplete_df[col_incomplete]
                complete_col = complete_df[col_incomplete]

                valid_mask = ~incomplete_col.isnull() & ~complete_col.isnull()

            if valid_mask.any():
                X = complete_col[valid_mask].values.reshape(-1, 1)
                y = incomplete_col[valid_mask].values.reshape(-1, 1)
                
                lr = LinearRegression()
                lr.fit(X, y)
                y_pred = lr.predict(X)
                
                r2 = r2_score(y, y_pred)
                r2_scores.append(r2)

                if np.isnan(r2):
                    continue
                
                fill_values.append(lr.predict(incomplete_col[~valid_mask].index.values.astype(float).reshape(-1, 1)))
                fill_results.append((incomplete_name, complete_name, col_incomplete))

    if r2_scores:
        best_idx = np.argmax(r2_scores)
        regressions.append(fill_values[best_idx])
        reg_names.append((incomplete_name, complete_name))

In [ ]:
# Print the name of the data frame and column used for filling NaN values
for result in fill_results:
    incomplete_name, complete_name, col_name = result
    print(f"Filled {col_name} in {incomplete_name} using {complete_name}")

In [ ]:
reg_names

In [ ]:
print(len(regressions))

In [ ]:
# Step 3: Use best regression to complete missing data
for incomplete_df, reg in regressions:
    columns_to_use = list(set(incomplete_df.columns) - {'datetime'})
    for column in columns_to_use:
        X = incomplete_df[column].values
        missing_mask = pd.isnull(X)
        print(missing_mask)
        if missing_mask.any():
            X_missing = X[missing_mask]
            print(type(X_missing))
            # incomplete_df.loc[missing_mask, column] = reg.predict(X_missing.reshape(-1, 1))

            # # Fill missing values with predictions
            # incomplete_df[column].fillna(value=pd.Series(reg.predict(X_missing.reshape(-1, 1)), index=incomplete_df.index[missing_mask]), inplace=True)

In [ ]:
#Step 3: For each incomplete df, fill in missing values using the best regression
for i,(incomplete_df, complete_df) in enumerate(regressions):
    # Use only columns that have complete data
    columns_to_use = list(set(complete_df.columns) & set(incomplete_df.columns))
    for col in columns_to_use:
        X=complete_df[col].values.reshape(-1,1)
        y=incomplete_df[col].values.reshape(-1,1)

        valid_mask = ~X.isna().any(axis=1) & ~y.isna().any(axis=1) # Get overlapping non-NaN values
        X_valid = X[valid_mask]
        y_valid = y[valid_mask]
        

        if len(y_valid) < 2: #if there are less than 2 valid values, skip
            continue

        reg = LinearRegression().fit(X_valid, y_valid)
        m = reg.coef_[0]
        b = reg.intercept_

        print(m,b)

    # # Fill in missing values for each column
    # for col in columns_to_use:
    #     nan_mask = incomplete_df[col].isna()
    #     print(incomplete_df.loc[nan_mask, col])# = m * complete_df.loc[nan_mask, col] + b

    # nan_mask= incomplete_df[columns_to_use].isna().any(axis=1)
    # print(incomplete_df.loc[nan_mask, columns_to_use])
    #incomplete_df.loc[nan_mask, columns_to_use] = m * complete_df.loc[nan_mask, columns_to_use] + b


    ###NOTE: this is not working because I need to do each column separately
    
    #incomplete_df[columns_to_use] = incomplete_df[columns_to_use].apply(lambda x: x.fillna(m * x + b), axis=1) #fill in missing values with regression line, axis=1 means apply to each row
    #dfs_imp = incomplete_df[columns_to_use].apply(lambda col: col.fillna(m * col + b), axis=1) 


    fig = make_subplots(rows=len(columns_to_use), cols=1)
    for i,r in enumerate(incomplete_df):
        fig.append_trace(go.Scatter(x=incomplete_df.index,y=incomplete_df[r],name="Incomplete_imputed"), row=i+1, col=1)
    
    # fig.show()
        
 

    

In [ ]:
### Step 2: Do linear regression with incomplete time series
regressions = []
for incomplete_df in incomplete_dfs:
    r2_scores = []
    for complete_df in complete_dfs:
        # Use only columns that have complete data
        columns_to_use = list(set(complete_df.columns) & set(incomplete_df.columns))

        X = complete_df[columns_to_use]
        y = incomplete_df[columns_to_use].reindex(X.index) # Reindex to make sure indices match
        
        if len(columns_to_use) < 2:
            continue

        regressors = []
        for column in columns_to_use:
            valid_mask = ~pd.isnull(X[column]) & ~pd.isnull(y[column])

            X_valid = X[valid_mask].values
            y_valid = y[valid_mask].values

            reg = LinearRegression().fit(X_valid, y_valid)
            r2_scores.append(r2_score(y_valid, reg.predict(X_valid)))
            regressors.append(reg)

    

#     # Choose regression with highest r2
#     best_reg = None
#     best_r2 = 0
#     for i, r2 in enumerate(r2_scores):
#         if r2 > best_r2:
#             best_reg = i
#             best_r2 = r2
    
#     print(best_r2,best_reg)
#     # regressions.append((incomplete_df, complete_dfs[best_reg]))

#         # if len(regressors) > 0:
#         #     best_reg_index = max(range(len(r2_scores)), key=r2_scores.__getitem__) # get index of regressor with highest r2 score
#         #     print(len(regressors))
#             #best_reg = regressors[best_reg_index]
#             # regressions.append((incomplete_df, best_reg))



In [ ]:
        # #Exploration plots of raw data
        # fig = make_subplots(rows=4, cols=1)
        # fig.append_trace(go.Scatter(x=soil_m.index,y=soil_m['S1'],name="Sensor1_original"), row=1, col=1)
        # fig.append_trace(go.Scatter(x=df_2.index,y=df_2['S1'],name="Sensor1_interpolate"), row=2, col=1)
        # fig.append_trace(go.Scatter(x=soil_m.index,y=soil_m['S2'],name="Sensor2_original"), row=3, col=1)
        # fig.append_trace(go.Scatter(x=df_2.index,y=df_2['S2'],name="Sensor2_int"), row=4, col=1)
        # fig.update_layout(height=900, width=1800, title_text="Soil_moisture"+'_'+tree+'_'+'original_2_sensors')
        # fig.show()


        # print(df_c)
        # fig = make_subplots(rows=6, cols=1)
        # fig.append_trace(go.Scatter(x=df_c.index,y=df_c['S1'],name="Sensor1_imp"), row=1, col=1)
        # fig.append_trace(go.Scatter(x=soil_m.index,y=soil_m['S1'],name="Sensor1_o"), row=2, col=1)
        # fig.append_trace(go.Scatter(x=df_c.index,y=df_c['S2'],name="Sensor2_imp"), row=3, col=1)
        # fig.append_trace(go.Scatter(x=soil_m.index,y=soil_m['S2'],name="Sensor2_o"), row=4, col=1)
        # fig.append_trace(go.Scatter(x=df_c.index,y=df_c['S3'],name="Sensor3_imp"), row=5, col=1)
        # fig.append_trace(go.Scatter(x=soil_m.index,y=soil_m['S3'],name="Sensor3_o"), row=6, col=1)
        # fig.update_layout(height=900, width=1800, title_text="Soil_moisture"+'_'+tree)
        # fig.show()
                

#### Extra 

In [ ]:
#EXAMPLES ON HOW TO USE .LOC

#slice 
#soil_m.loc['2021-05-01 02:00:00':'2021-10-21 11:00:00','S1':'S2'] 

#boolean array 
#soil_m.loc[:,[True,False,True]]

#type(soil_m.loc['2021-05-01 02:00:00',['S1']])

#n_=soil_m.loc[:]!=0
#print(n_.all())

#df = pd.DataFrame([[1, 2], [4, 5], [7, 8]],
#     index=['cobra', 'viper', 'sidewinder'],
#     columns=['max_speed', 'shield'])

#notation 
#df.loc[:,'max_speed'] #returns a pd.series 
#df.loc[:,['max_speed']] #returns a data frame

# Alignable index
#df.loc[pd.Index(["cobra", "viper"], name="foo")]
#df.loc[pd.Index(["viper", "cobra"])]

#Alignable series
#df.loc[pd.Series([False, True, False], index=['viper', 'sidewinder', 'cobra'])]

#condition returns a boolean Series with column labels specified
#df.loc[df['shield']>6,'max_speed']